#Kraken Project
##Erfan Mahmoudinia 97212889

In this project we are going to implement Kraken which is refrenced below . As students suggested in piazza we are using ncbi-genome-download to create index file dynamically. we used ete3 package which have taxonomy tree. which we can call a node or family to get the genomes. this process help us to handle ram and storage we need for this project.



Refrence : 
https://genomebiology.biomedcentral.com/articles/10.1186/gb-2014-15-3-r46

In [1]:
!pip install ncbi-genome-download
!pip install ete3
!wget ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/assembly_summary.txt

     |████████████████████████████████| 10.5MB 8.2MB/s 
  Created wheel for ete3: filename=ete3-3.1.1-cp36-none-any.whl size=2226265 sha256=789db2112c9bc8dfe3f862d7e77868035d00b4112e2ddabebffbd3f4b01f4cee
  Stored in directory: /root/.cache/pip/wheels/14/90/50/5c10d965e7fb68343d9c5349cce2787ff3e38397946468d640
Successfully built ete3
--2020-02-05 20:01:36--  ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/assembly_summary.txt
           => ‘assembly_summary.txt’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 2607:f220:41e:250::13
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/genbank/bacteria ... done.
==> SIZE assembly_summary.txt ... 156865997
==> PASV ... done.    ==> RETR assembly_summary.txt ... done.
Length: 156865997 (150M) (unauthoritative)

assembly_summary.tx 100%[===================>]

The assembly_summary.txt contain information about the bacteria existed in ncbi database this file is corresponds to the tree we will use from ete3 package. we need assembly_summary file to get taxid of each bacteria. as you can see we show the head of this file.

In [2]:
import pandas as pd
summary = pd.read_csv("assembly_summary.txt",sep="\t",skiprows=1)
summary.head()



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material
0,GCA_000010525.1,PRJDA19267,SAMD00060925,NaN,representative genome,438753,7,Azorhizobium caulinodans ORS 571,strain=ORS 571,NaN,latest,Complete Genome,Major,Full,2007/10/16,ASM1052v1,University of Tokyo,GCF_000010525.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,assembly from type material
1,GCA_000007365.1,PRJNA312,SAMN02604269,NaN,representative genome,198804,9,Buchnera aphidicola str. Sg (Schizaphis graminum),strain=Sg,NaN,latest,Complete Genome,Major,Full,2002/07/02,ASM736v1,Uppsala Univ.,GCF_000007365.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN
2,GCA_000007725.1,PRJNA256,SAMN02604289,NaN,representative genome,224915,9,Buchnera aphidicola str. Bp (Baizongia pistaciae),strain=Bp (Baizongia pistaciae),NaN,latest,Complete Genome,Major,Full,2003/01/29,ASM772v1,Valencia Univ.,GCF_000007725.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN
3,GCA_000009605.1,PRJNA245,SAMD00061095,NaN,reference genome,107806,9,Buchnera aphidicola str. APS (Acyrthosiphon pi...,NaN,Tokyo1998,latest,Complete Genome,Major,Full,2004/05/11,ASM960v1,Rikken GSC,GCF_000009605.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN
4,GCA_000021065.1,PRJNA31223,SAMN02604133,NaN,na,561501,9,Buchnera aphidicola str. Tuc7 (Acyrthosiphon p...,strain=Tuc7,NaN,latest,Complete Genome,Major,Full,2009/01/06,ASM2106v1,University of Arizona,GCF_000021065.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN


For all of the following processes we need tax id mapped to each file name using summary file

In [0]:
def file_to_taxId(filename):
  splitted = filename[0:15]
  return int(summary[summary['gbrs_paired_asm'] == splitted]["taxid"])
  
  

In [4]:
file_to_taxId("GCF_000014005.1_ASM1400v1_genomic.fasta.gz")

204669

NCBITaxa contains the tree we talked in the previous section. which we load it in ncbi variable to use in the following processes

In [5]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

NCBI database not present yet (first time used?)
Done. Parsing...


Loading node names...
2222991 names loaded.
214593 synonyms loaded.
Loading nodes...
2222991 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /root/.etetoolkit/taxa.sqlite ...
 2222000 generating entries... 
Uploading to /root/.etetoolkit/taxa.sqlite


Inserting synonyms:      25000 

Inserting synonyms:      210000 

Inserting taxids:       25000 

Inserting taxids:       2220000 

here we used get_name_translator to choose the part of the tree that corresponds to bacteria name we give the function then we write the leaves of the sub tree into my_taids.txt

In [0]:
bacteria = "acidobacteria"

subTree= ncbi.get_name_translator([bacteria])[bacteria][0]
z = ncbi.get_descendant_taxa(subTree, return_tree=True)

f = open('my_taxids.txt', 'w+')
for pp in z.get_leaves():
    f.write(pp.name + '\n')
f.close()

Now with using my_taxids.txt we can download the complete genome of this bacteria

In [0]:
!ncbi-genome-download --format fasta --taxid my_taxids.txt --assembly-level complete bacteria

And now we create genomes folder to integrate the data into one folder to run the following processes in a easy way.

In [8]:
%%shell
mkdir genomes
for f in $(find refseq/ -maxdepth 3 -type f); do
b=$(basename $f)
if [ ${b: -7} == ".fna.gz" ]; then  
mv  "$f" "/content/genomes/${b%.fna.gz}.fasta.gz"
fi  
done

To read fasta and fastq files we need to use biopython package , which installed with pip install

In [9]:
!pip install biopython

     |████████████████████████████████| 2.3MB 8.2MB/s 


here we set the parameters such as kmer_len and minimizer_len for the consistancy.

In [0]:
Kmer_len = 31
Minimizer_len = 13

definition Reverse Complement Code Which Reverse the sequence and then complement it. complement means 

A --> T

T --> A

C --> G

G --> C


In [0]:
def Reverse_Complement(seq):
  result = ''
  for ch in seq[len(seq)::-1]:
    if ch == 'A':
      result = result + 'T'
    elif ch == 'T':
      result = result + 'A'
    elif ch == 'C':
      result = result + 'G'
    elif ch == 'G':
      result = result + 'C'
  return result


we use defaultdict in order to create a dictionary of dictionaries. 

In [0]:
from collections import defaultdict


Minimizer Finder is a function which consider Z as an initial minimizer which will be replaced by the first minimizer. 

in this function we get a sequnce(kmer) as an input then run a window in this sequence with length of minimizer length that user defined.

and finally do the same for the reverse complement.


In [0]:
def Minimizer_finder(seq):
  minimizer_optimal = 100000000
  position = -1
  seq_rev = Reverse_Complement(seq)
  for j in range(len(seq)-Minimizer_len):
    minimizer = hash(seq[j:j+Minimizer_len])
    minimizer_rev = hash(seq_rev[j:j+Minimizer_len])
    if minimizer < minimizer_optimal:
      minimizer_optimal = minimizer
      position = j
    if minimizer_rev < minimizer_optimal:
      minimizer_optimal = minimizer_rev
      position = len(seq) - j - Minimizer_len  
  return minimizer_optimal , position

The Core of the project is the next section which I named it sequence analyzer. this functin tend to find kmer and correspond minimizer using previous function. 

I designed this function to run optimizely as it run when the last minimizer is done. it means that we just check the last charecters to check if any new minimizer created or not! and check if current minimizer goes out from the window. 


hash function : I used python hash function which is consistant between systems and this hash function cause the uniform distribution (approximatly) and ease our work when we want to find the indeces.

In [0]:
Minimizer_Kmer = defaultdict(dict)

def seq_anlyzer(seq , taxId):
  previous_minimizer = 1000000000
  minimizer_position = -1
  for i in range(len(seq)-Kmer_len):
    kmer = seq[i:i+Kmer_len]
    if minimizer_position < 0:
      previous_minimizer , minimizer_position = Minimizer_finder(kmer)
    else:
      if hash(kmer[Kmer_len-Minimizer_len:Kmer_len]) < previous_minimizer: 
        previous_minimizer , minimizer_position = hash(kmer[Kmer_len-Minimizer_len:Kmer_len]) , Kmer_len-Minimizer_len
      if  hash(Reverse_Complement(kmer[Kmer_len-Minimizer_len:Kmer_len])) < previous_minimizer:
        previous_minimizer , minimizer_position = hash(Reverse_Complement(kmer[Kmer_len-Minimizer_len:Kmer_len])) , Kmer_len-Minimizer_len
    if kmer in Minimizer_Kmer[previous_minimizer]:
      if taxId != Minimizer_Kmer[previous_minimizer][kmer]:
        Minimizer_Kmer[previous_minimizer][kmer] = z.get_common_ancestor([Minimizer_Kmer[previous_minimizer][kmer] , taxId]).name
    else: 
      Minimizer_Kmer[previous_minimizer][kmer] = taxId
    minimizer_position -= 1
      

This part is just for checking the functions 

In [21]:
seq = "ACCGATGCATCGATCGATCGACTGACTGACTAGCTTCGATCGATCGTACGTACGTGTCAGTCA"
seq_anlyzer(seq , "240015")
print(seq)
print(seq[len(seq)::-1])
print(Reverse_Complement(seq))
# Minimizer_finder(seq)

ACCGATGCATCGATCGATCGACTGACTGACTAGCTTCGATCGATCGTACGTACGTGTCAGTCA
ACTGACTGTGCATGCATGCTAGCTAGCTTCGATCAGTCAGTCAGCTAGCTAGCTACGTAGCCA
TGACTGACACGTACGTACGATCGATCGAAGCTAGTCAGTCAGTCGATCGATCGATGCATCGGT


You can see Minimizer_Kmer Dictionary this structure is like below

In [22]:
Minimizer_Kmer

defaultdict(dict,
            {-9101192026733300387: {'ACTAGCTTCGATCGATCGTACGTACGTGTCA': '240015',
              'ACTGACTAGCTTCGATCGATCGTACGTACGT': '240015',
              'ACTGACTGACTAGCTTCGATCGATCGTACGT': '240015',
              'AGCTTCGATCGATCGTACGTACGTGTCAGTC': '240015',
              'ATCGACTGACTGACTAGCTTCGATCGATCGT': '240015',
              'CGACTGACTGACTAGCTTCGATCGATCGTAC': '240015',
              'CTAGCTTCGATCGATCGTACGTACGTGTCAG': '240015',
              'CTGACTAGCTTCGATCGATCGTACGTACGTG': '240015',
              'CTGACTGACTAGCTTCGATCGATCGTACGTA': '240015',
              'GACTAGCTTCGATCGATCGTACGTACGTGTC': '240015',
              'GACTGACTAGCTTCGATCGATCGTACGTACG': '240015',
              'GACTGACTGACTAGCTTCGATCGATCGTACG': '240015',
              'GATCGACTGACTGACTAGCTTCGATCGATCG': '240015',
              'TAGCTTCGATCGATCGTACGTACGTGTCAGT': '240015',
              'TCGACTGACTGACTAGCTTCGATCGATCGTA': '240015',
              'TGACTAGCTTCGATCGATCGTACGTACGTGT': '240015',
              'T

Now it's time to read the files which we save them in genomes folder and push it into seq_analyzer function in order to create index file. 

In [23]:
from Bio import SeqIO
import os
import gzip
genomes_files = os.listdir("genomes/")


Minimizer_Kmer = defaultdict(dict)

index = 1

for genome_file in genomes_files:
  with gzip.open("genomes/" + genome_file, "rt") as handle:
    fasta_sequences = SeqIO.parse(handle,'fasta')
    taxid = file_to_taxId(genome_file)
    print("======================================")
    print("filename : " , genome_file ," Genome Number : " , index  , " taxid : " , taxid)
    for fasta in fasta_sequences:
        name, sequence = fasta.id, str(fasta.seq)
        seq_anlyzer(sequence , str(taxid))
    index += 1
    print("======================================")
      

filename :  GCF_000014905.1_ASM1490v1_genomic.fasta.gz  Genome Number :  1  taxid :  234267
filename :  GCF_000178975.2_ASM17897v2_genomic.fasta.gz  Genome Number :  2  taxid :  1198114
filename :  GCF_000178955.2_ASM17895v2_genomic.fasta.gz  Genome Number :  3  taxid :  682795
filename :  GCF_000014005.1_ASM1400v1_genomic.fasta.gz  Genome Number :  4  taxid :  204669
filename :  GCF_000265425.1_ASM26542v1_genomic.fasta.gz  Genome Number :  5  taxid :  926566
filename :  GCF_000179915.2_ASM17991v2_genomic.fasta.gz  Genome Number :  6  taxid :  401053
filename :  GCF_003330725.1_ASM333072v1_genomic.fasta.gz  Genome Number :  7  taxid :  2211140
filename :  GCF_000226295.1_ASM22629v1_genomic.fasta.gz  Genome Number :  8  taxid :  981222
filename :  GCF_001618865.1_ASM161886v1_genomic.fasta.gz  Genome Number :  9  taxid :  1855912
filename :  GCF_008000815.1_ASM800081v1_genomic.fasta.gz  Genome Number :  10  taxid :  1592106
filename :  GCF_000022565.1_ASM2256v1_genomic.fasta.gz  Genome N

find_score is the function which we can calculate the score of each kmer. actullay associate each kmer by using index file. accosiation between kmer and taxids.

In [0]:

def find_score(seq):
  previous_minimizer = 100000000
  minimizer_position = -1
  for i in range(len(seq)-Kmer_len):
    kmer = seq[i:i+Kmer_len]
    if minimizer_position < 0:
      previous_minimizer , minimizer_position = Minimizer_finder(kmer)
    else:
      if hash(kmer[Kmer_len-Minimizer_len:Kmer_len]) < previous_minimizer: 
        previous_minimizer , minimizer_position = hash(kmer[Kmer_len-Minimizer_len:Kmer_len]) , Kmer_len-Minimizer_len
      if hash(Reverse_Complement(kmer[Kmer_len-Minimizer_len:Kmer_len])) < previous_minimizer:
        previous_minimizer , minimizer_position = hash(Reverse_Complement(kmer[Kmer_len-Minimizer_len:Kmer_len])) , Kmer_len-Minimizer_len

    if kmer in Minimizer_Kmer[previous_minimizer]:
      if Minimizer_Kmer[previous_minimizer][kmer] not in scores:
        scores[Minimizer_Kmer[previous_minimizer][kmer]] = 0
      scores[Minimizer_Kmer[previous_minimizer][kmer]] += 1
    minimizer_position -= 1


Then we create a sub topolpogy using ete3 package from the nodes we used. and calculate Root to leaves (RTL) for each node to find the best match.

In [26]:
import operator

association = dict()

index = 0 
with open("GCA_000014005.1_ASM1400v1_genomic.fna_R1.fastq", "rt") as handle:
    for record in SeqIO.parse(handle, "fastq"):
      print("=============================================")
      print("number : " , index)
      scores = dict()
      find_score(record.seq)
      if len(scores) > 0 :
        tree = ncbi.get_topology(list(scores.keys()))
        main_scores = dict()
        for taxid in list(scores.keys()):
          score = 0 
          node = tree.search_nodes(name=str(taxid))[0]
          ancestors = node.get_ancestors()
          for ancestor in ancestors:
            score += scores[ancestor.name]
          main_scores[str(taxid)] = score
        maxAssociated = max(main_scores.items(), key=operator.itemgetter(1))[0]
        if maxAssociated not in association:
          association[maxAssociated] = 0
        association[maxAssociated] += 1
      index += 1
  

number :  0


/usr/local/lib/python3.6/dist-packages/Bio/Seq.py:182: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  BiopythonWarning,


number :  1
number :  2
number :  3
number :  4
number :  5
number :  6
number :  7
number :  8
number :  9
number :  10
number :  11
number :  12
number :  13
number :  14
number :  15
number :  16
number :  17
number :  18
number :  19
number :  20
number :  21
number :  22
number :  23
number :  24
number :  25
number :  26
number :  27
number :  28
number :  29
number :  30
number :  31
number :  32
number :  33
number :  34
number :  35
number :  36
number :  37
number :  38
number :  39
number :  40
number :  41
number :  42
number :  43
number :  44
number :  45
number :  46
number :  47
number :  48
number :  49
number :  50
number :  51
number :  52
number :  53
number :  54
number :  55
number :  56
number :  57
number :  58
number :  59
number :  60
number :  61
number :  62
number :  63
number :  64
number :  65
number :  66
number :  67
number :  68
number :  69
number :  70
number :  71
number :  72
number :  73
number :  74
number :  75
number :  76
number :  77
number :

Here is the number of association to each taxid

In [27]:
association

{'204434': 1, '204669': 4998}

and the contribution in each taxid calculated as below

In [0]:
Contribution = {k: v / total for total in (sum(association.values()),) for k, v in association.items()}

In [29]:
Contribution

{'204434': 0.00020004000800160032, '204669': 0.9997999599919984}